In [ ]:
import numpy as np
import gdspy
import matplotlib.pyplot as plt

%matplotlib inline
from functools import partial

In [ ]:
def plot_cell(cell):
    # FROM: https://github.com/heitzmann/gdspy/issues/42
    poly_dict = cell.get_polygons(by_spec=True)
    plt.figure(figsize=(20, 20))
    for layer_datatype, polys in poly_dict.items():
        if layer_datatype[0] == 2:
            continue
        for poly in polys:
            plt.fill(*poly.T, lw=0.5, ec="r", fc="none")

In [ ]:
TRENCH_LAYER = 2
FEEDING_CHANNEL_LAYER = 6

In [ ]:
coverslip_dims = np.array([55e3, 24e3])
chip_dims = np.array([23e3, 13e3])
outline_thickness = 0.15e3
chip_margin = 1e3
port_margin = chip_margin / 2
trench_width = 1.5
trench_length = 35
lane_gap = 20
effective_trench_length = trench_length + lane_gap / 2
trench_spacing = 2
feeding_channel_width = 90  # 300#90
port_radius = 200
inner_snake_turn_radius = effective_trench_length
outer_snake_turn_radius = feeding_channel_width + inner_snake_turn_radius
lane_fc_dims = np.array(
    [chip_dims[0] - 2 * chip_margin - outer_snake_turn_radius, feeding_channel_width]
)
lane_height = feeding_channel_width + 2 * effective_trench_length
num_lanes = (chip_dims[1] - 2 * chip_margin) // lane_height
if num_lanes % 2 == 0:
    num_lanes -= 1  # ensure odd
last_lane_y = ((num_lanes - 1) * lane_height) / 2

In [ ]:
Cell = partial(gdspy.Cell, exclude_from_current=True)

In [ ]:
lib = gdspy.GdsLibrary()

In [ ]:
# feedingchannel_cell = Cell('Feeding Channel')
port_cell = Cell("Feeding Channel Port")
trench_cell = Cell("Trench")
lane_cell = Cell("Lane")
snake_cell = Cell("Snake")  # use instead of chip_cell

In [ ]:
chip_cell = Cell("Chip")
chip_outline_inner = gdspy.Rectangle(-chip_dims / 2, chip_dims / 2)
chip_outline_outer = gdspy.Rectangle(
    -(chip_dims + outline_thickness) / 2, (chip_dims + outline_thickness) / 2
)
chip_outline = gdspy.fast_boolean(chip_outline_outer, chip_outline_inner, "not")
chip_cell.add(chip_outline)

In [ ]:
lane_fc = gdspy.Rectangle(-lane_fc_dims / 2, lane_fc_dims / 2)
lane_cell.add(lane_fc)

In [ ]:
lane_ys = np.linspace(-last_lane_y, last_lane_y, num_lanes)
for y in lane_ys:
    chip_cell.add(gdspy.CellReference(lane_cell, (0, y)))

In [ ]:
bend_cell = Cell("Feeding Channel Bend")
bend = gdspy.Round(
    (0, 0),
    outer_snake_turn_radius,
    inner_radius=inner_snake_turn_radius,
    number_of_points=500,
)
bend = gdspy.slice(bend, 0, 0)
bend_cell.add(bend[1])

In [ ]:
for y in lane_ys[1::2]:
    chip_cell.add(
        gdspy.CellReference(bend_cell, (lane_fc_dims[0] / 2, y + lane_height / 2))
    )
    chip_cell.add(
        gdspy.CellReference(
            bend_cell, (-lane_fc_dims[0] / 2, y - lane_height / 2), rotation=180
        )
    )

In [ ]:
port_x = chip_dims[0] / 2 - lane_fc_dims[0] / 2 - port_radius - port_margin
port = gdspy.Round((port_x, 0), port_radius, number_of_points=500)
port_fc = gdspy.Rectangle((0, -lane_fc_dims[1] / 2), (port_x, lane_fc_dims[1] / 2))
port_cell.add(port)
port_cell.add(port_fc)

In [ ]:
chip_cell.add(
    gdspy.CellReference(port_cell, (-lane_fc_dims[0] / 2, lane_ys[-1]), rotation=180)
)
chip_cell.add(gdspy.CellReference(port_cell, (lane_fc_dims[0] / 2, lane_ys[0])))

In [ ]:
main_cell = Cell("main")
main_cell.add(gdspy.CellReference(chip_cell, (0, 0)))

In [ ]:
plot_cell(main_cell)

In [ ]:
lib = gdspy.GdsLibrary(
    infile="/Users/jacob/Dropbox/Research/Paulsson/snakenbake/test/20170426_Vibrio_Suction_W1.GDS"
)

In [ ]:
[c.name for c in lib.top_level()]

In [ ]:
main_cell = next(c for c in lib.top_level() if c.name == "main")
poly_dict = main_cell.get_polygons(by_spec=True)

In [ ]:
plt.figure(figsize=(20, 20))
for layer_datatype, polys in poly_dict.items():
    if layer_datatype[0] == 2:
        continue
    for poly in polys:
        plt.fill(*poly.T, lw=0.5, ec="r", fc="none")